In [15]:
import h5py
from pyfaidx import Fasta
from pathlib import Path
import pandas as pd

In [28]:
fasta_1 = "../../data/3FTx/3FTx_mapped.fasta"
fasta_2 = "../../data/Pla2g2/Pla2g2.fasta"

emb_prott5_1 = "../../data/3FTx/emb_prott5.h5"
emb_prott5_2 = "../../data/Pla2g2/emb_prott5.h5"
emb_esm2_1 = "../../data/3FTx/emb_esm2.h5"
emb_esm2_2 = "../../data/Pla2g2/emb_esm2.h5"

out_dir = Path("../../data/Pla2g2_3FTx/")
out_fasta = out_dir / "seq.fasta"
out_csv = out_dir / "features.csv"
out_emb_esm2 = out_dir / "emb_esm2.h5"
out_emb_prott5 = out_dir / "emb_prott5.h5"

# Sequences FASTA

In [20]:
f1 = Fasta(fasta_1)
f2 = Fasta(fasta_2)

csv_data = []
with open(out_fasta, "w") as handler:
    for header, seq in f1.items():
        handler.write(f">{header}\n")
        handler.write(f"{seq}\n")
        csv_data.append([header, "3FTx"])
    for header, seq in f2.items():
        handler.write(f">{header}\n")
        handler.write(f"{seq}\n")
        csv_data.append([header, "Pla2g2"])

# Feature CSV

In [23]:
df = pd.DataFrame(csv_data, columns=["uid", "dset"])
df.to_csv(out_csv, index=False)

# Embeddings H5

In [32]:
import numpy as np

In [44]:
emb_data = {}

with (
    h5py.File(emb_prott5_1, "r") as hdf_t5_1,
    h5py.File(emb_prott5_2, "r") as hdf_t5_2,
    h5py.File(emb_esm2_1, "r") as hdf_esm_1,
    h5py.File(emb_esm2_2, "r") as hdf_esm_2,
    h5py.File(out_emb_prott5, "w") as out_t5,
    h5py.File(out_emb_esm2, "w") as out_esm2,
    ):
    for header, emb in hdf_t5_1.items():
        out_t5.create_dataset(name=header, data=emb)
    for header, emb in hdf_t5_2.items():
        out_t5.create_dataset(name=header, data=emb)
    for header, emb in hdf_esm_1.items():
        out_esm2.create_dataset(name=header, data=emb)
    for header, emb in hdf_esm_2.items():
        out_esm2.create_dataset(name=header, data=emb)

In [43]:
with (
    h5py.File(out_emb_prott5, "r") as out_t5,
    h5py.File(out_emb_esm2, "r") as out_esm2,
    ):
    for header, emb in out_t5.items():
        if emb.shape == (2560,):
            print(emb)
            break
    for header, emb in out_esm2.items():
        if emb.shape != (2560,):
            print(emb)
            break
        # print(header, emb)
        # print(emb.shape)
        # break